# Run All HMS 4.13 Example Projects

This notebook demonstrates comprehensive execution of all example projects and runs
included with HEC-HMS 4.13.

**Projects:** castro, river_bend, tenk, tifton

For each project, we will:
1. Extract it from the samples.zip
2. Parse all available runs from the .run file
3. Execute each run via direct Java invocation
4. Track success/failure and execution time

## Setup

### Installation Options

**Option A: Install from PyPI** (recommended for users)
```bash
pip install hms-commander
```

**Option B: Development Mode** (for contributors)  
Run the second code cell below instead of the first.

In [1]:
# =============================================================================
# OPTION A: PyPI Installation (Default)
# =============================================================================
# Uncomment the line below if hms-commander is not installed:
# !pip install --upgrade hms-commander

import re
import time
from pathlib import Path
from datetime import datetime

from hms_commander import HmsExamples, HmsJython, __version__

print(f"hms-commander v{__version__}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

hms-commander v0.1.0
Started: 2026-01-05 01:37:24


In [2]:
# =============================================================================
# OPTION B: Development Mode (Local Copy)
# =============================================================================
# Uncomment ALL lines below and run INSTEAD of Option A above

# import sys
# import re
# import time
# from pathlib import Path
# from datetime import datetime
#
# # Prioritize local development copy
# hmscmdr_directory = Path.cwd().parent
# if str(hmscmdr_directory) not in sys.path:
#     sys.path.insert(0, str(hmscmdr_directory))
#
# print(f"Loading hms-commander from: {hmscmdr_directory}")
#
# from hms_commander import HmsExamples, HmsJython, __version__
#
# print(f"hms-commander v{__version__}")
# print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Configuration

In [3]:
# HMS version to test
HMS_VERSION = "4.13"

# Output directory for extracted projects (isolated folder for this notebook)
OUTPUT_BASE = Path.cwd() / 'example_projects' / 'hms413_run_all'
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

# Get HMS executable path
hms_exe = HmsExamples.get_hms_exe(HMS_VERSION)
print(f"HMS Version: {HMS_VERSION}")
print(f"HMS Executable: {hms_exe}")
print(f"Output Directory: {OUTPUT_BASE}")

2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.10 at C:\Program Files\HEC\HEC-HMS\4.10
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.11 at C:\Program Files\HEC\HEC-HMS\4.11
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.12 at C:\Program Files\HEC\HEC-HMS\4.12
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.13 at C:\Program Files\HEC\HEC-HMS\4.13
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.4.1 at C:\Program Files\HEC\HEC-HMS\4.4.1
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.5 at C:\Program Files\HEC\HEC-HMS\4.5
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.6 at C:\Program Files\HEC\HEC-HMS\4.6
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.7.1 at C:\Program Files\HEC\HEC-HMS\4.7.1
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Found HMS 4.8 at C:\Program Files\HEC\HEC-HMS\4.8
2026-01-05 01:37:24 -

HMS Version: 4.13
HMS Executable: C:\Program Files\HEC\HEC-HMS\4.13\HEC-HMS.cmd
Output Directory: c:\GH\hms-commander\examples\example_projects\hms413_run_all


## List Available Projects

In [4]:
# Get all projects for this version
all_projects = HmsExamples.list_projects()
projects = all_projects.get(HMS_VERSION, [])

print(f"Projects in HMS {HMS_VERSION}:")
for p in projects:
    print(f"  - {p}")
print(f"\nTotal: {len(projects)} projects")

2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Catalog built: 68 project entries


Projects in HMS 4.13:
  - castro
  - river_bend
  - tenk
  - tifton

Total: 4 projects


## Helper Function: Parse Runs from Project

In [5]:
def get_runs_from_project(project_path: Path) -> list:
    """
    Parse all run names from the .run file in a project.
    
    The .run file contains entries like:
        Run: Run Name Here
        ...
    End:
    
    Returns:
        List of run names
    """
    runs = []
    
    # Find .run file
    run_files = list(project_path.glob('*.run'))
    if not run_files:
        print(f"  WARNING: No .run file found in {project_path}")
        return runs
    
    run_file = run_files[0]
    content = run_file.read_text(encoding='utf-8', errors='ignore')
    
    # Parse run names
    for match in re.finditer(r'^Run:\s*(.+)$', content, re.MULTILINE):
        run_name = match.group(1).strip()
        if run_name:
            runs.append(run_name)
    
    return runs

print("Helper function defined.")

Helper function defined.


## Extract Projects and Discover Runs

In [6]:
# Extract all projects and discover runs
project_info = {}

for project_name in projects:
    print(f"\nExtracting: {project_name}")
    
    try:
        # Extract project
        project_path = HmsExamples.extract_project(
            project_name,
            version=HMS_VERSION,
            output_path=OUTPUT_BASE / project_name
        )
        
        # Get runs
        runs = get_runs_from_project(project_path)
        
        project_info[project_name] = {
            'path': project_path,
            'runs': runs
        }
        
        print(f"  Path: {project_path}")
        print(f"  Runs: {runs}")
        
    except Exception as e:
        print(f"  ERROR: {e}")
        project_info[project_name] = {'path': None, 'runs': [], 'error': str(e)}

# Summary
total_runs = sum(len(p['runs']) for p in project_info.values())
print(f"\n{'='*60}")
print(f"Total: {len(project_info)} projects, {total_runs} runs")

2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Removing existing project folder: c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Extracting 'castro' from HMS 4.13
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Source: C:\Program Files\HEC\HEC-HMS\4.13\samples.zip
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Destination: c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Successfully extracted 'castro' to c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Removing existing project folder: c:\GH\hms-commander\examples\example_projects\hms413_run_all\river_bend\river_bend
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Extracting 'river_bend' from HMS 4.13
2026-01-05 01:37:24 - hms_commander.HmsExamples - I


Extracting: castro
  Path: c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro
  Runs: ['Current', 'Future']

Extracting: river_bend
  Path: c:\GH\hms-commander\examples\example_projects\hms413_run_all\river_bend\river_bend
  Runs: ['Minimum Facility', 'Minimum Facility + Pump', 'Minimum Facility + 4Pumps']

Extracting: tenk


2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Successfully extracted 'tenk' to c:\GH\hms-commander\examples\example_projects\hms413_run_all\tenk\tenk
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Removing existing project folder: c:\GH\hms-commander\examples\example_projects\hms413_run_all\tifton\tifton
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Extracting 'tifton' from HMS 4.13
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Source: C:\Program Files\HEC\HEC-HMS\4.13\samples.zip
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Destination: c:\GH\hms-commander\examples\example_projects\hms413_run_all\tifton\tifton
2026-01-05 01:37:24 - hms_commander.HmsExamples - INFO - Successfully extracted 'tifton' to c:\GH\hms-commander\examples\example_projects\hms413_run_all\tifton\tifton


  Path: c:\GH\hms-commander\examples\example_projects\hms413_run_all\tenk\tenk
  Runs: ['Jan 96 storm']

Extracting: tifton
  Path: c:\GH\hms-commander\examples\example_projects\hms413_run_all\tifton\tifton
  Runs: ['1970 simulation']

Total: 4 projects, 7 runs


## Execute All Runs

In [7]:
# Execute all runs and track results
results = []
run_count = 0
total_runs = sum(len(p['runs']) for p in project_info.values() if p.get('path'))

print(f"Executing {total_runs} runs across {len(project_info)} projects...")
print("="*70)

for project_name, info in project_info.items():
    if not info.get('path'):
        continue
        
    project_path = info['path']
    
    for run_name in info['runs']:
        run_count += 1
        print(f"\n[{run_count}/{total_runs}] {project_name} / {run_name}")
        
        # Generate script
        script = HmsJython.generate_compute_script(
            project_path=project_path,
            run_name=run_name,
            save_project=True
        )
        
        # Execute
        start_time = time.time()
        try:
            success, stdout, stderr = HmsJython.execute_script(
                script_content=script,
                hms_exe_path=hms_exe,
                working_dir=project_path,
                timeout=300,
                max_memory="4G"
            )
            elapsed = time.time() - start_time
            
            # Check for success indicators
            actual_success = (
                ('Computation completed' in stdout) or 
                (success and 'Error' not in stderr)
            )
            
            status = "SUCCESS" if actual_success else "FAILED"
            error_msg = stderr[:200] if not actual_success and stderr else None
            
        except Exception as e:
            elapsed = time.time() - start_time
            actual_success = False
            status = "ERROR"
            error_msg = str(e)[:200]
            stdout = ""
        
        # Record result
        results.append({
            'project': project_name,
            'run': run_name,
            'success': actual_success,
            'time': elapsed,
            'error': error_msg
        })
        
        print(f"  {status} ({elapsed:.1f}s)")
        if error_msg:
            print(f"  Error: {error_msg[:100]}...")

print(f"\n{'='*70}")
print("Execution complete!")

2026-01-05 01:37:24 - hms_commander.HmsJython - INFO - Executing HMS 4.13 via direct Java invocation
2026-01-05 01:37:24 - hms_commander.HmsJython - INFO - Script: c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro\hms_script.py
2026-01-05 01:37:24 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


Executing 7 runs across 4 projects...

[1/7] castro / Current


2026-01-05 01:37:28 - hms_commander.HmsJython - INFO - HMS 4.13 script executed successfully
2026-01-05 01:37:28 - hms_commander.HmsJython - INFO - Executing HMS 4.13 via direct Java invocation
2026-01-05 01:37:28 - hms_commander.HmsJython - INFO - Script: c:\GH\hms-commander\examples\example_projects\hms413_run_all\castro\castro\hms_script.py
2026-01-05 01:37:28 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


  SUCCESS (3.8s)

[2/7] castro / Future


2026-01-05 01:37:31 - hms_commander.HmsJython - INFO - HMS 4.13 script executed successfully
2026-01-05 01:37:31 - hms_commander.HmsJython - INFO - Executing HMS 4.13 via direct Java invocation
2026-01-05 01:37:31 - hms_commander.HmsJython - INFO - Script: c:\GH\hms-commander\examples\example_projects\hms413_run_all\river_bend\river_bend\hms_script.py
2026-01-05 01:37:31 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


...

  SUCCESS (132.0s)

[6/7] tenk / Jan 96 storm


2026-01-05 01:43:52 - hms_commander.HmsJython - INFO - HMS 4.13 script executed successfully
2026-01-05 01:43:52 - hms_commander.HmsJython - INFO - Executing HMS 4.13 via direct Java invocation
2026-01-05 01:43:52 - hms_commander.HmsJython - INFO - Script: c:\GH\hms-commander\examples\example_projects\hms413_run_all\tifton\tifton\hms_script.py
2026-01-05 01:43:52 - hms_commander.HmsJython - INFO - Memory: -Xms128M -Xmx4G


  SUCCESS (3.4s)

[7/7] tifton / 1970 simulation


2026-01-05 01:43:55 - hms_commander.HmsJython - INFO - HMS 4.13 script executed successfully


  SUCCESS (2.9s)

Execution complete!


## Results Summary

In [8]:
import pandas as pd

# Create results DataFrame
df = pd.DataFrame(results)

# Summary statistics
total = len(df)
passed = df['success'].sum()
failed = total - passed
total_time = df['time'].sum()

print(f"HMS {HMS_VERSION} Example Projects - Execution Summary")
print("="*60)
print(f"Total Runs:  {total}")
print(f"Passed:      {passed} ({100*passed/total:.0f}%)")
print(f"Failed:      {failed} ({100*failed/total:.0f}%)")
print(f"Total Time:  {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"Avg Time:    {total_time/total:.1f}s per run")

# Per-project summary
print(f"\n{'='*60}")
print("Results by Project:")
print("-"*60)

project_summary = df.groupby('project').agg({
    'success': ['sum', 'count'],
    'time': 'sum'
}).round(1)
project_summary.columns = ['Passed', 'Total', 'Time (s)']
project_summary['Status'] = project_summary.apply(
    lambda r: 'OK' if r['Passed'] == r['Total'] else 'ISSUES', axis=1
)
print(project_summary.to_string())

HMS 4.13 Example Projects - Execution Summary
Total Runs:  7
Passed:      7 (100%)
Failed:      0 (0%)
Total Time:  390.2s (6.5 min)
Avg Time:    55.7s per run

Results by Project:
------------------------------------------------------------
            Passed  Total  Time (s) Status
project                                   
castro           2      2       6.7     OK
river_bend       3      3     377.1     OK
tenk             1      1       3.4     OK
tifton           1      1       2.9     OK


In [9]:
# Detailed results table
print("\nDetailed Results:")
print("-"*70)

display_df = df[['project', 'run', 'success', 'time']].copy()
display_df['status'] = display_df['success'].apply(lambda x: 'PASS' if x else 'FAIL')
display_df['time'] = display_df['time'].apply(lambda x: f"{x:.1f}s")
display_df = display_df[['project', 'run', 'status', 'time']]

print(display_df.to_string(index=False))


Detailed Results:
----------------------------------------------------------------------
   project                       run status   time
    castro                   Current   PASS   3.8s
    castro                    Future   PASS   2.9s
river_bend          Minimum Facility   PASS 114.8s
river_bend   Minimum Facility + Pump   PASS 130.3s
river_bend Minimum Facility + 4Pumps   PASS 132.0s
      tenk              Jan 96 storm   PASS   3.4s
    tifton           1970 simulation   PASS   2.9s


In [10]:
# Show failed runs if any
failed_df = df[~df['success']]

if len(failed_df) > 0:
    print(f"\n{'='*60}")
    print(f"FAILED RUNS ({len(failed_df)}):")
    print("-"*60)
    
    for _, row in failed_df.iterrows():
        print(f"\nProject: {row['project']}")
        print(f"Run: {row['run']}")
        if row['error']:
            print(f"Error: {row['error']}")
else:
    print(f"\n{'='*60}")
    print("ALL RUNS PASSED!")
    print("="*60)


ALL RUNS PASSED!


## Cleanup (Optional)

In [11]:
# Uncomment to clean up extracted projects
# import shutil
# if OUTPUT_BASE.exists():
#     shutil.rmtree(OUTPUT_BASE)
#     print(f"Cleaned up: {OUTPUT_BASE}")

## Summary

This notebook executed all simulation runs from all example projects included with HMS 4.13:

| Project | Description |
|---------|-------------|
| **castro** | Castro Valley, CA - urban watershed |
| **river_bend** | River Bend example |
| **tenk** | Ten Mile Creek example |
| **tifton** | Little River near Tifton, GA |

### Key Points

- **Direct Java invocation** bypasses batch file bugs
- **Memory configurable** via `max_memory` parameter
- Run names parsed from `.run` files
- Results tracked with timing information

### Using This Pattern

```python
# Extract project
project_path = HmsExamples.extract_project('tifton', version='4.13')

# Generate and execute script
script = HmsJython.generate_compute_script(project_path, run_name)
success, stdout, stderr = HmsJython.execute_script(
    script, hms_exe, max_memory="8G"
)
```